# Discrete-time approximations of continuous-time systems

Often, controllers are designed for continuous-time systems, yet have to be implemented on a computer operating at discrete time steps. This raises the question of how derivatives in continuous-time can be approximated using discrete-time samples. It is important to realize that the manner of this discretization can have important effects and consequences on the dynamics of the controlled system as a whole, as will be demonstrated by this notebook.

### Errors from discretization

We first examine how discretizing derivatives can introduce errors between the true evolution of a continuous-time system and its discretized approximation. To this end, we generate a simple cosine wave described by the differential equation
\begin{equation}
\ddot{x}(t) = -\omega^2 x(t)
\end{equation}
or, in matrix form:
\begin{equation}
\begin{bmatrix} \dot{x} \\ \ddot{x} \end{bmatrix} = \begin{bmatrix} 0 & 1 \\ -\omega^2 & 0 \end{bmatrix}\begin{bmatrix} x \\ \dot{x}\end{bmatrix}.
\end{equation}
For comparison, we demonstrate the results obtained by approximating the derivative with a forward Euler scheme and by the midpoint method using various sampling time steps $h > 0$.

First, let us briefly review these two methods. Let a continuous time system evolved according to the equation:
\begin{equation}
\dot{x}(t) = f(x(t))
\end{equation}
The forward Euler scheme is defined by the approximation of derivatives using a sampling time $h$ as:
\begin{equation}
 \dot{x}(t) \approx \dfrac{x(t+h) - x(t)}{h},
\end{equation}
or, using operator notation:
\begin{equation}
 \dot{x}(t) \approx \dfrac{q - 1}{h}x(t),
\end{equation}
where $q$ denotes the operation of a forward time shift by $h$. Substituting into the system dynamics, this leads to the following relation between sampled values in consecutive time steps:
\begin{equation}
x_{k+1} = x_{k} + h \dot{x}_ {k} = x_{k} + h f(x_k)
\end{equation}
This is known as the forward Euler method.

A slightly better approximation can be obtained by stating that the slope evaluated using a finite difference method actually approximates the true slope at the midpoint as:
\begin{equation}
 \dot{x}\left(t + \frac{h}{2}\right) \approx \dfrac{x(t+h) - x(t)}{h}
\end{equation}
which leads to the so-called midpoint method:
\begin{equation}
x_{k+1} = x_{k} + h f(x_{k+1/2}) = x_{k} + h f(x_{k} + \frac{h}{2}f(x_{k})),
\end{equation}
where $x_{k+1/2}$ has been evaluated according to a forward Euler step with step-size $h/2$.

The following code compares the generated cosine waves with each method. Note that both forward-type discretization schemes yield an unstable wave, though the midpoint method diverges at a slower rate. However, the midpoint method requires twice as many $f(\cdot)$ derivative evaluations.

In [1]:
using Interact, Plots

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17445085182314510151\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-3111350785738066615\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17445085182314510151\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
# function describing the system dynamics (the derivative of the state x)
function f(x, ω)
    return [0.0 1.0; -ω^2 0.0] * x
end

# forward Euler method
function simulateForwardEuler(T, Δt, ω)
    NT = Int(round(T/Δt))
    x = zeros(2, NT+1)
    x[:,1] = [1.0; 0.0]
    for k = 1:NT
        x[:,k+1] = x[:, k] .+ Δt .* f(x[:,k], ω)
    end
    return x, NT
end

# midpoint method
function simulateMidpoint(T, Δt, ω)
    NT = Int(round(T/Δt))
    x = zeros(2, NT+1)
    xkhalf = zeros(2)
    x[:,1] = [1.0; 0]
    for k = 1:NT
        xkhalf .= x[:,k] .+ (Δt/2).*f(x[:,k],ω)
        x[:,k+1] = x[:,k] .+ Δt.*f(xkhalf, ω)
    end
    return x, NT
end;    

In [3]:
@manipulate for Δt=widget(0.001:0.005:0.2, label="Δt", labelcolor=:red), ω=widget(0.5:0.01:1.5, label="ω"), T = OrderedDict("T=10" => 10, "T=20" => 20, "T=50" => 50), toggle = widget([:auto, :fixed], label="y-axis")
    x, NT = simulateForwardEuler(T, Δt, ω)
    x2, NT = simulateMidpoint(T, Δt, ω)
    xTrue = cos.(collect(T*(0:NT)/NT).*ω)
    vTrue = .-ω * sin.(collect(T*(0:NT)/NT).*ω)
    l = @layout [a b]
    ylims = toggle==:auto ? :auto : (-1.5, 1.5)
    p1 = plot(T*(0:NT)/NT, view(x, 1, :), linewidth=2, ylims=ylims, label="",linetype=:steppost)
    plot!(p1, T*(0:NT)/NT, view(x2, 1, :), linewidth=2, label="",linetype=:steppost)
    plot!(p1, T*(0:NT)/NT, xTrue, linewidth=3, label="")
     toggle==:auto ? :auto : (-2.0, 2.0)
    p2 = plot(T*(0:NT)/NT, view(x, 2, :), linewidth=2, ylims=ylims, label="",linetype=:steppost, legend=:outerright)
    plot!(p2, T*(0:NT)/NT, view(x2, 2, :), linewidth=2, label="",linetype=:steppost)
    plot!(p2, T*(0:NT)/NT, vTrue, linewidth=3, label="")
    vbox(
        plot(p1, p2, size=(1000, 300), layout=l))
    end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Δt"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 40,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:labelcolor => :red,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
20, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/tusike/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/tusike/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/tusike/.julia/packages/InteractBase/sOe2Z/src/../assets/all.js"), Asset("css", nothing, "/home/tusike/.julia/packages/InteractBase/sOe2Z/src/../assets/style.css"), Asset("css", nothing, "/home/tusike/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007ff2131db340, Task (runnable) @0x00007ff2131db340), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.001\",\"0.006\",\"0.011\",\"0.016\",\"0.021\",\"0.026\",\"0.031\",\"0.036\",\"0.041\",\"0.046\",\"0.051\",\"0.056\",\"0.061\",\"0.066\",\"0.071\",\"0.076\",\"0.081\",\"0.086\",\"0.091\",\"0.096\",\"0.101\",\"0.106\",\"0.111\"